In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from composite_functions import fb_evolution_composites
from advection_rd_ml import advection_rd_ml
from vertical_advection_rd_ml import vertical_advection_rd_ml

In [2]:
def mld_avg_var(mld,var,z):
    zmask= (z < mld) # z above mixed layer
    var=var.convert_calendar("proleptic_gregorian",use_cftime=False)
    intfield=var.where(zmask, drop=True).mean('lev').squeeze()
    return intfield

In [3]:
def sub_ML_avg_var(mld,var,z):
    zmask= (z > mld) # z below mixed layer
    #print(zmask)
    var=var.convert_calendar("proleptic_gregorian",use_cftime=False)
    fld=var.where(zmask, drop=False).squeeze()
    i=(fld.notnull().argmax(dim='lev'))
    fldsub=fld.isel(lev=i)
    return fldsub

In [4]:
def calc_qpen(SW,MLD):
    eu=2.71818
    T1=0.58*eu**(-MLD/0.35)
    T2=0.42*eu**(-MLD/23)
    qpen=SW*(T1+T2)
    return qpen

In [5]:
timesfut=["2070-01","2100-01"]
timespi=["1850-01","1880-01"]
tipast=["1850-01","2015-01"]
tissp=["2015-01","2100-01"]
n3=[-5,5,210,270] #Nino3
n34=[-5,5,190,240] # nino3.4?
n4=[-5,5,160,210]
pac=[-5,5,120,280]
regbox=[-40,40,90,300]

In [6]:
ds_sst=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/tos_1850_2100_E3SMv1LE.nc")
tos=ds_sst['tos']
time=ds_sst['time']

In [7]:
dsThist=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/thetao_2070_2100_E3SMv1LE.nc")
dsWhist=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/wo_2070_2100_E3SMv1LE.nc")
dsUhist=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/uo_2070_2100_E3SMv1LE.nc")
dsVhist=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/vo_2070_2100_E3SMv1LE.nc")
dsHF=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/hfds_1850_2100_E3SMv1LE.nc")
dsSW=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/rsds_1850_2100_E3SMv1LE.nc")
dsML=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/mlotst_1850_2100_E3SMv1LE.nc")
dsS=xr.open_dataset("/glade/work/smagahey/UCSB/codes/generated_files/sos_1850_2100_E3SMv1LE.nc")

In [8]:
Th=dsThist['thetao'].sel(lat=slice(n34[0],n34[1]),lon=slice(pac[2],pac[3])) #.mean('lat').mean('lon')
Wh=dsWhist['wo'].sel(lat=slice(n34[0],n34[1]),lon=slice(pac[2],pac[3])) #.mean('lat').mean('lon')
Uh=dsUhist['uo'].sel(lat=slice(n34[0],n34[1]),lon=slice(pac[2],pac[3])) #.mean('lat').mean('lon')
Vh=dsVhist['vo'].sel(lat=slice(n34[0],n34[1]),lon=slice(pac[2],pac[3]))
HF=dsHF['hfds'].sel(lat=slice(n34[0],n34[1]),lon=slice(pac[2],pac[3]),time=slice(timesfut[0],timesfut[1]))
SW=dsSW['rsds'].sel(lat=slice(n34[0],n34[1]),lon=slice(pac[2],pac[3]),time=slice(timesfut[0],timesfut[1]))
MLD=dsML['mlotst'].sel(lat=slice(n34[0],n34[1]),lon=slice(pac[2],pac[3]),time=slice(timesfut[0],timesfut[1]))
sal=dsS['sos'].sel(lat=slice(n34[0],n34[1]),lon=slice(pac[2],pac[3]),time=slice(timesfut[0],timesfut[1]))

In [9]:
lat=dsML['lat']
lon=dsML['lon']

In [10]:
Qpen=calc_qpen(SW,MLD[1:,:,:,:])
THFLX=HF[1:,:,:,:]-Qpen
TH=THFLX/(3850*1025*MLD[1:,:,:,:])

In [11]:
MLD=MLD.convert_calendar("proleptic_gregorian",use_cftime=False)

In [12]:
int_T=mld_avg_var(MLD[1:,:,:,:],Th,Th.lev)
int_U=mld_avg_var(MLD[1:,:,:,:],Uh,Uh.lev)
int_V=mld_avg_var(MLD[1:,:,:,:],Vh,Vh.lev)
int_W=mld_avg_var(MLD[1:,:,:,:],Wh,Wh.lev)

In [13]:
# get temps, velocities below ML
sub_int_T=sub_ML_avg_var(MLD[1:,:,:,:],Th,Th.lev)
sub_int_U=sub_ML_avg_var(MLD[1:,:,:,:],Uh,Uh.lev)
sub_int_V=sub_ML_avg_var(MLD[1:,:,:,:],Vh,Vh.lev)
sub_int_W=sub_ML_avg_var(MLD[1:,:,:,:],Wh,Wh.lev)

In [14]:
mnadv_climU,mnadv_climV,ekman,T1,h_eddy,dSpdt=advection_rd_ml(sal,lat,lon,time,int_U,int_V,1,)

In [15]:
up,thermo,eddy,updTpbar=vertical_advection_rd_ml(MLD[1:,:,:,:],lat,lon,int_T,int_U,int_V,sub_int_W,sub_int_T)

tm_mld shape= (15, 12, 10, 160)
wtmp shape= (15, 359, 9, 159, 12)


In [16]:
upwelling=up.drop_vars('lev')*86400*30.41
thermocline=thermo.drop_vars('lev')*86400*30.41
Q=TH*86400*30.41
eddyV=eddy.drop_vars('lev')*86400*30.41
updT=updTpbar*86400*30.41
ekman=ekman*86400*30.41
T1=T1*86400*30.41
eddyH=h_eddy*86400*30.41

In [17]:
Q=Q.convert_calendar("proleptic_gregorian",use_cftime=False)
T1=T1.convert_calendar("proleptic_gregorian",use_cftime=False)

In [18]:
temptend= Q - T1 - ekman - eddyH - upwelling - thermocline - eddyV

In [19]:
hb_data=xr.Dataset({'temp_tend':temptend,'Q':Q,'umdTpdx':T1,'ekman':ekman,'eddyH':eddyH,'upwelling':upwelling,'thermocline':thermocline,'eddyV':eddyV})
hb_data.to_netcdf('/glade/work/smagahey/UCSB/codes/ML_heat_budget/python_hb/hist_hb_terms.nc')